In [48]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./mnist/data/', one_hot = True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


- 영문자 4개로 구성된 단어를 학습시켜 3글자만 주어지면 나머지한 글자를 추천하여 단어를 완성하는 프로그램

In [49]:
char_arr = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split(' ')
num_dic = {n : i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [50]:
# 학습에 사용할 단어를 배열로 저장
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

- 유틸리티 함수 작성
    - 입력값으로, 단어의 처음 세 글자의 알파벳 인덱스를 구한 배열을구한다.
    - inputs = [num_dict[n] for n in seq[:-1]]
    - 출력값용으로, 마지막 글자의 알파벳 인덱스를 구한다
    - outputs = num_dict[seq[-1]
    - 입력값을 원-핫 인코딩으로 변환
    - input_batch.append(np.eye(dic_len([input])
 

In [51]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    for seq in seq_data:
        inputs = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[inputs])
        target_batch.append([target])
    return input_batch, np.array(target_batch).reshape([-1])

In [52]:
make_batch(seq_data)

([array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0., 

In [53]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

#단어 중 처음 3글자를 단계적으로 학습
n_step = 3
n_input = n_class = dic_len

In [54]:
with tf.device('/cpu:0'):
    X = tf.placeholder(tf.float32, [None, n_step, n_input])
    Y = tf.placeholder(tf.int32, [None])

    W = tf.Variable(tf.random_normal([n_hidden, n_class]))
    b = tf.Variable(tf.random_normal([n_class]))

In [55]:
with tf.device('/cpu:0'):
    cell1 = tf.contrib.rnn.BasicLSTMCell(n_hidden)
    cell1 = tf.contrib.rnn.DropoutWrapper(cell1, output_keep_prob=0.5)
    cell2 = tf.contrib.rnn.BasicLSTMCell(n_hidden)
    # 위의 셀 2개를 MultiRNNcell로 조합
    multi_cell = tf.contrib.rnn.MultiRNNCell([cell1, cell2])

    outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [56]:
with tf.device('/cpu:0'):
    outputs = tf.transpose(outputs, [1, 0, 2])
    outputs = outputs[-1]
    model = tf.matmul(outputs, W) + b

In [57]:
with tf.device('/cpu:0'):
    cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits = model))
    train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    prediction =  tf.cast(tf.argmax(tf.nn.softmax(model), 1), tf.int32)
    is_correct = tf.equal(Y, prediction)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [59]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([train, cost], feed_dict = {X: input_batch, Y: target_batch})
    print('Epoch: {}, cost : {}'.format(epoch, loss))
    predict, accuracy_val = sess.run([prediction, accuracy], feed_dict = {X: input_batch, Y: target_batch})    
    print('prediction : {}, accuracy : {}'.format(predict, accuracy_val))
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict = {X: input_batch, Y: target_batch})    
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
print('\n\n -----예측 결과 =====')
print('입력값 :', [w[:3] + ' 'for w in seq_data])
print('출력값 :', predict_words)
print('정확도:', accuracy_val)


Epoch: 0, cost : 3.469670534133911
prediction : [ 3  3 17  3  3  3  3  3  3  3], accuracy : 0.5
Epoch: 1, cost : 2.6267683506011963
prediction : [3 3 4 3 3 3 3 3 3 3], accuracy : 0.5
Epoch: 2, cost : 1.5942342281341553
prediction : [3 3 4 4 3 3 3 3 3 3], accuracy : 0.6000000238418579
Epoch: 3, cost : 0.8662815093994141
prediction : [ 3  3 15  4  3  3  4  4 18  3], accuracy : 0.800000011920929
Epoch: 4, cost : 0.5175931453704834
prediction : [ 3  3 15 15 11 11  3  4 18  3], accuracy : 0.800000011920929
Epoch: 5, cost : 0.6146663427352905
prediction : [ 3  3 15  4  3  3  3  3 18  3], accuracy : 0.800000011920929
Epoch: 6, cost : 0.5510315895080566
prediction : [ 3  3 15  4 11 11  3  3 18  3], accuracy : 0.800000011920929
Epoch: 7, cost : 0.5230320692062378
prediction : [ 3  3 15  4 11 11  3  4 18  3], accuracy : 0.8999999761581421
Epoch: 8, cost : 0.18682286143302917
prediction : [ 3  3 15  4  3  3  4  4 18  3], accuracy : 0.800000011920929
Epoch: 9, cost : 0.4864080548286438
prediction 

In [47]:
tf.reset_default_graph()
sess.close()